In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

import muspy
from pathlib import Path

In [2]:
song = muspy.read('../midi/examples/bitmidi/pink-panther.mid')
song

Music(metadata=Metadata(schema_version='0.0', source_filename='pink-panther.mid', source_format='midi'), resolution=240, tempos=[Tempo(time=0, qpm=125.0)], time_signatures=[TimeSignature(time=0, numerator=4, denominator=4)], tracks=[Track(program=0, is_drum=False, name='Melody', notes=[Note(time=640, pitch=56, duration=80, velocity=100), Note(time=720, pitch=57, duration=80, velocity=100), Note(time=880, pitch=58, duration=80, velocity=100), ...]), Track(program=0, is_drum=False, name='Piano Accomp', notes=[Note(time=640, pitch=37, duration=80, velocity=100), Note(time=640, pitch=44, duration=80, velocity=100), Note(time=640, pitch=49, duration=80, velocity=100), ...]), Track(program=0, is_drum=False, name='Alto Sax', notes=[Note(time=46960, pitch=51, duration=77, velocity=112), Note(time=47040, pitch=52, duration=77, velocity=112), Note(time=47440, pitch=54, duration=77, velocity=112), ...]), ...])

In [3]:
song.tracks[0].notes[-1]

Note(time=85200, pitch=78, duration=2160, velocity=115)

In [4]:

def extract_melodies(song: muspy.Music, song_name: str or int, set_name: str, length_in_bars: int):
    track = get_melody_track(song.tracks)

    if track is not None:
        is_call = True
        count = 1

        # bar_dur_in_sec = 60 / song.tempos[0].qpm * song.time_signature[0].nominator * 4 / song.time_signature[0].denominator
        time_steps_per_bar = song.resolution * song.time_signatures[0].numerator * 4 / song.time_signatures[0].denominator
        starting_bar = round(track.notes[0].time / time_steps_per_bar)

        starting_time = starting_bar * time_steps_per_bar
        time_marker_start = starting_time
        time_marker_end = time_marker_start + time_steps_per_bar * length_in_bars

        base_path = '../data/reference_data/' + set_name + '/' + str(song_name) + '/'
        Path(base_path).mkdir(parents=True, exist_ok=True)

        excerpt = muspy.Music()
        excerpt.resolution = song.resolution
        excerpt.tempos.append(song.tempos[0])
        excerpt.time_signatures.append(song.time_signatures[0])
        excerpt_track = muspy.Track()
        excerpt_track.name = 'melody'
        excerpt.append(excerpt_track)

        for note in track.notes:
            if (note.time < starting_time):
                pass
            elif (note.time < time_marker_end):
                note.time = note.time - time_marker_start
                excerpt_track.append(note)
            else:
                # save excerpt
                if is_call: 
                    muspy.write_midi(base_path + '{:02d}'.format(count) + '_call.mid', excerpt, backend='pretty_midi')
                else:
                    muspy.write_midi(base_path + '{:02d}'.format(count) + '_response.mid', excerpt, backend='pretty_midi')
                # init new one
                excerpt = muspy.Music()
                excerpt.resolution = song.resolution
                excerpt.tempos.append(song.tempos[0])
                excerpt.time_signatures.append(song.time_signatures[0])
                excerpt_track = muspy.Track()
                excerpt_track.name = 'melody'
                excerpt.append(excerpt_track)
                # change time markers (move window x bars)
                time_marker_start = time_marker_end
                time_marker_end = time_marker_start + time_steps_per_bar * length_in_bars
                # increase count and toggle call/response
                if not is_call:
                    count += 1
                is_call = not is_call


def get_melody_track(tracks):
    for track in tracks:
        if track.name.lower() == "melody" or track.name.lower() == "vocals" or track.name.lower() == "mel":
            return track
    else:
        return None



In [5]:
extract_melodies(song, 'pink_panther', 'test_set', 4)